# Problema 3: la forma di `M`

In breve, nel Notebook originale `M` è diagonale ma ha degli array come elementi, con l'idea che i nodi sorgente racchiudano sia i pesi sia il bias del layer (aka nodo interno) corrispondente.

Se però volessimo distinguere questi input?

## Setup

Facciamo veloce, che tanto oramai li conosciamo

In [1]:
using LinearAlgebra, Revise

Revise.includet(joinpath(@__DIR__,"..","Operator.jl"))

using .MyOperator

***OSS***: nel modulo `.Operator` ci sono più metodi di quelli presenti nell'articolo originale!

### La rete neurale

Riusiamo le celle di codice già viste...

In [2]:
using OffsetArrays

# funzione di attivazione
h(x)  = tanh(x)
h′(x) = 1 - h(x)^2


# questa funzione, di fatto, effettua il forward pass
function neural_net(params,X₀;h=h,h′= h′)
    T = Matrix{Float64}
    N = length(params)
    X = OffsetArray(Vector{T}(undef,N+1),0:N)   
    Δ = Vector{T}(undef, N)
    X[0] = X₀
    W = first.(params)
    B = last.(params)
    
    for i=1:N         
          X[i] =  h.(W[i]*X[i-1] .+ B[i])
          Δ[i] =  h′.(W[i]*X[i-1] .+ B[i])        
    end 
    X,Δ
end

neural_net (generic function with 1 method)

Adesso possiamo costruire la rete

In [3]:
## Un po' di parametri che descrivono la rete...

n = [5,4,3,1]   # contiene [n₀...n_N], le dimensioni dei layer
k = 10          # batchsize
N = length(n)-1 # numero di layer (nascosti + output). Dev'essere positivo

init(sizes...) = 0.01randn(sizes...)    # utility function per inizializzare

init (generic function with 1 method)

In [4]:
## Un po' di parametri degni di questo nome

# Creiamo le matrici dei pesi e i vettori dei bias
Ws_and_bs =[ [init(n[i+1],n[i]) , init(n[i+1])]  for i=1:N] # The second part of the pair is a vector here

X₀ = init(n[1],k)         # batch di k pattern (aka i dati)
y  = init(n[end],k);      # y is what we will compare X_N against (aka l'etichetta)

𝓁(x,y)  = sum(abs2,x-y)/2   # loss (errore quadratico)
𝓁′(x,y) = x .- y;           # derivata della loss (w.r.t. output layer)

X, δ = neural_net(Ws_and_bs,X₀) # Inferenza (aka forward pass)
# X e δ hanno i valori per ogni layer (servono per fare il backward pass)

([[-0.0006933712885586175 -0.022213780060294074 … 0.0022010053899360767 -0.008681635089007331; 0.01371033744276951 0.005627602989682013 … -0.000821467707027095 0.0025277126427506714; … ; -0.010273994914017084 -0.0017402676438274683 … 0.0033771064851509554 0.001475814749556125; 0.008511043430941185 0.006449493358035305 … -0.0037750029360778857 0.012244473849873119], [-0.001359465729245611 -0.0011193388674030763 … -0.0012247078943883826 -0.0012615332522085338; -0.02027198835690988 -0.020056942374687983 … -0.020575428981007206 -0.02019399845374865; -0.00046475243507199896 -0.000468882128447815 … 0.00013007153195079187 -9.49077005642371e-5; 0.0016449962407624396 0.0012475814275313814 … 0.0014518258489282514 0.0015360566883672604], [0.000443964983575913 0.00044273146189499824 … 0.0004398702932028956 0.0004372766188392979; -0.017709892167667267 -0.017712176010874287 … -0.01770864632234514 -0.01770925582385858; 0.017912702215229385 0.017908589260831633 … 0.01790824779824713 0.0179095960255095

Adesso possiamo calcolare il gradiente della loss usando la nostra tecnica

In [5]:
## The diagonal matrix
#M = Diagonal([ [ℋ(δ[i]) ∘ ℛ(X[i-1])  ℋ(δ[i])] for i=1:N]) # OLD!
M = Diagonal([ [ℋ(δ[i]) ∘ ℛ(X[i-1])  ℋ(δ[i]) ∘ ℛ(ones(1,k))] for i=1:N])

#M = Bidiagonal([ℋ(δ[i]) ∘ ℛ(X[i-1]) for i=1:N], [ℋ(δ[i]) ∘ ℛ(ones(1,k)) for i=1:(N-1)], :U)
#M = [M [fill(𝒪, N-1); ℋ(δ[N]) ∘ ℛ(ones(1,k))]]


## The lower triangular matrix (I-L)
ImL = Bidiagonal([ℐ() for i in 1:N], -[ℋ(δ[i]) ∘ ℒ(Ws_and_bs[i][1]) for i=2:N] , :L)

## derivata della loss (rispetto all'output layer)
g = [ fill(𝒪(),N-1) ; [𝓁′(X[N],y)] ]      

## Finalmente, il gradiente della loss, usando il backslash
∇J = M' * (ImL' \ g)

3-element Vector{Matrix{Matrix{Float64}}}:
 [[2.921837886888091e-8 -1.4711433974972633e-8 … 8.321687123215068e-9 3.786571328199305e-11; -1.394753445563111e-7 7.022635947221968e-8 … -3.9723331733844425e-8 -1.79618643558014e-10; -1.0866779572546805e-7 5.4714128932074444e-8 … -3.0949713876235533e-8 -1.4088794553047937e-10; -5.025187034600354e-8 2.5301714711811154e-8 … -1.4312244643745392e-8 -6.51813557114257e-11]; [-3.725536599527891e-6; 1.778386428063789e-5; 1.3855866276020647e-5; 6.4074454460771365e-6;;];;]
 [[1.4021156294797213e-6 2.4043593639447166e-5 1.2832992243825e-7 -1.6034115944457495e-6; 8.006226231062926e-7 1.3729142259427515e-5 7.327771836586497e-8 -9.155647215308128e-7; -7.703036664355318e-7 -1.3209230271742555e-5 -7.050273756158094e-8 8.808929712731839e-7]; [-0.0011800264282396267; -0.0006738073744923615; 0.0006482908098094921;;];;]
 [[-4.290091726418208e-5 0.0017225307999695642 -0.0017417754785848884]; [-0.09726197995251608;;];;]

## Indagine del problema

Vediamo com'è fatta `M`

In [6]:
M

3×3 Diagonal{Matrix{Main.MyOperator.Operator}, Vector{Matrix{Main.MyOperator.Operator}}}:
 [ℋ(4, 10)∘ℛ(5, 10) ℋ(4, 10)∘ℛ(1, 10)]  …    ⋅  
   ⋅                                         ⋅  
   ⋅                                       [ℋ(1, 10)∘ℛ(3, 10) ℋ(1, 10)∘ℛ(1, 10)]

`M` è di tipo `Diagonal`. In posizione $(i,i)$ ha i parametri del layer $i$-esimo: sia i pesi, sia il bias.

Nell'articolo originale, ciascun nodo interno del grafo computazionale aveva due nodi entranti: uno corrispondente ai suoi parametri, l'altro corrispondente al layer precedente. 

Però avrebbe senso anche dividere i pesi e i bias in due famiglie distinte di nodi (come ho fatto in presentazione). 

Come diventa la `M` in questo caso? E nel codice che succede?

In [7]:
# Prima versione della nuova M

Mnuova = Matrix{eltype(ImL)}(undef, 3,6)

for i = 1:size(Mnuova,1), j = 1:size(Mnuova,2)
    Mnuova[i, j] = 𝒪()
    Mnuova[i, 2i-1] = M[i,i][1]
    Mnuova[i, 2i] = M[i,i][2]
end

display(Mnuova)

3×6 Matrix{Main.MyOperator.Operator}:
 ℋ(4, 10)∘ℛ(5, 10)  ℋ(4, 10)∘ℛ(1, 10)  …  𝒪
 𝒪                  𝒪                     𝒪
 𝒪                  𝒪                     ℋ(1, 10)∘ℛ(1, 10)

`M` è sempre diagonale, ma a blocchi. I pesi e i bias sono in due elementi diversi, stavolta.

In [8]:
x = (ImL' \ g)
for v in x
    display(v)
end

4×10 Matrix{Float64}:
 -4.02431e-7  -7.69835e-7  -5.68909e-7  …  -7.54559e-7  -4.86312e-7
  1.9218e-6    3.67633e-6   2.71681e-6      3.60338e-6   2.32237e-6
  1.4967e-6    2.86314e-6   2.11586e-6      2.80632e-6   1.80867e-6
  6.9213e-7    1.32402e-6   9.7845e-7       1.29774e-6   8.36394e-7

3×10 Matrix{Float64}:
 -0.000127466  -0.000243837  -0.000180196  …  -0.000238999  -0.000154034
 -7.28072e-5   -0.000139277  -0.000102926     -0.000136513  -8.79827e-5
  7.00505e-5    0.000134004   9.90289e-5       0.000131345   8.46515e-5

1×10 Matrix{Float64}:
 -0.0105083  -0.020102  -0.0148554  …  0.0105197  -0.0197031  -0.0126986

In [12]:
try
    Mnuova' * x
catch e
    showerror(stderr, e)
end

DimensionMismatch: arrays could not be broadcast to a common size: a has axes Base.OneTo(3) and b has axes Base.OneTo(4)

Il problema sono gli zeri. Consideriamo ad esempio la prima riga

In [14]:
display(Mnuova'[1,1] * x[1])
display(Mnuova'[1,2] * x[2])
display(Mnuova'[1,3] * x[3])

4×5 Matrix{Float64}:
  2.92184e-8  -1.47114e-8   8.89709e-9   8.32169e-9   3.78657e-11
 -1.39475e-7   7.02264e-8  -4.24682e-8  -3.97233e-8  -1.79619e-10
 -1.08668e-7   5.47141e-8  -3.30898e-8  -3.09497e-8  -1.40888e-10
 -5.02519e-8   2.53017e-8  -1.53019e-8  -1.43122e-8  -6.51814e-11

3×10 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

1×10 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

Queste tre matrici andrebbero sommate, per andare a formare il primo elemento di `∇J`. 
- Chiaramente, non sono compatibili
- Eppure, due di questi addendi sono zeri
Quindi la somma la si potrebbe (anzi, dovrebbe) evitare!

Nel codice originale degli autori, il problema viene evitato ricorrendo a un "trucco". Essendo la `M` diagonale, nel fare `M' * x` non vengono fatte somme! Certo, `M[i,i] * x[i]` coinvolge (l'aggiunta di) una `Matrix{Operator}`, ma questo viene preso in carico da una regola apposta

In matematica siamo spesso tentati (e spesso abituati) ad ignorare gli zeri, o a considerarli come "un' unico zero indistinto". E con buona ragione. Eppure è una questione assai rilevante, che spesso fa da "campanello d'allarme" per una ambiguità ancora più grossa.

Comunque, a parte la filosofia, nella pratica ha molta importanza la dimensione dello zero, essendoci matrici...

Per poter svolgere questo calcolo si potrebbe, ad esempio:
- Trattare lo zero come un "simbolo", ovvero non valutarlo mai, e sovraccaricare le operazioni in maniera opportuna
- Valutare l'omomorfismo nullo, ma fare un qualche `reshape`. Il problema è dove
- Aggirare il problema, come hanno fatto gli autori